<a href="https://colab.research.google.com/github/crystalloide/RAG/blob/main/LAB43_Protocoles_communication_entre_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LAB43 : Implémentation de protocoles de communication entre agents :

**Objectif:** Concevoir et appliquer des protocoles structurés de communication agent-à-agent (schémas + états + routage).

**Durée estimée:** 15–20 minutes

**Livrable:** Un notebook Python où plusieurs agents échangent des messages sous des protocoles validés (Request/Reply, Pub/Sub, et Negotiate/Contract).

---

## 1) Setup :

Installation des dépendances nécessaires et configuration de l'API OpenAI.

In [1]:
# Installation des packages
!pip install -q pydantic openai python-dotenv

In [2]:
# Configuration de l'API Key OpenAI
import os
from google.colab import userdata

# Option 1: Utiliser les secrets Colab (recommandé)
try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
    print("✓ Clé API chargée depuis les secrets Colab")
except:
    # Option 2: Saisie manuelle
    from getpass import getpass
    OPENAI_API_KEY = getpass('Entrez votre clé API OpenAI: ')
    os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
    print("✓ Clé API configurée manuellement")

✓ Clé API chargée depuis les secrets Colab


## 2) Core Types: schémas des messages + Bus

Définition des types de base pour les messages et le bus de communication.

In [3]:
from __future__ import annotations
from typing import Literal, List, Dict, Any, Optional
from dataclasses import dataclass, field
from pydantic import BaseModel, Field, ValidationError
from time import time
import uuid
import json
from openai import OpenAI

# Initialisation du client OpenAI
llm = OpenAI()

# --- Message schema (validated envelope) ---
class Msg(BaseModel):
    id: str = Field(default_factory=lambda: str(uuid.uuid4()))
    proto: Literal["REQREP", "PUBSUB", "NEGOTIATE"]
    type: str                       # e.g., REQUEST, REPLY, PROPOSE, ACCEPT...
    sender: str
    to: List[str] = Field(default_factory=list)
    topic: Optional[str] = None     # for PUBSUB
    corr_id: Optional[str] = None   # correlation for replies
    payload: Dict[str, Any] = Field(default_factory=dict)
    ts: float = Field(default_factory=time)

print("✓ Classe Msg définie")

✓ Classe Msg définie


In [4]:
# --- Simple in-memory bus with mailboxes ---
@dataclass
class Mailbox:
    name: str
    inbox: List[Msg] = field(default_factory=list)

class Bus:
    def __init__(self):
        self.boxes: Dict[str, Mailbox] = {}

    def register(self, *names):
        for n in names:
            if n not in self.boxes:
                self.boxes[n] = Mailbox(n)

    def send(self, msg: Msg):
        if msg.proto == "PUBSUB":
            # broadcast to all subscribers of topic; here: all except sender
            for name, box in self.boxes.items():
                if name != msg.sender:
                    box.inbox.append(msg)
        else:
            for dest in msg.to:
                self.boxes[dest].inbox.append(msg)

    def recv(self, name: str, max_items: int = 10) -> List[Msg]:
        box = self.boxes[name]
        out, box.inbox = box.inbox[:max_items], box.inbox[max_items:]
        return out

# Créer une instance globale du bus
bus = Bus()
print("✓ Bus de messages créé")

✓ Bus de messages créé


## 3) Prompts pour les rôles & Helper to Talk :

Définition des rôles et fonction helper pour communiquer avec le LLM.

In [5]:
def chat_as(role: str, system: str, user: str, temp=0):
    """Helper pour générer une réponse LLM selon un rôle spécifique."""
    r = llm.chat.completions.create(
        model="gpt-4o-mini",
        temperature=temp,
        messages=[
            {"role": "system", "content": f"Role: {role}. {system}"},
            {"role": "user", "content": user}
        ]
    )
    return r.choices[0].message.content.strip()

# Définition des rôles
ROLES = {
    "Requester": "You create clear task specs and validate replies.",
    "Worker": "You fulfill requests precisely and report status.",
    "Auditor": "You observe traffic and flag malformed or unsafe messages."
}

print("✓ Fonction chat_as et rôles définis")
print(f"  Rôles disponibles: {', '.join(ROLES.keys())}")

✓ Fonction chat_as et rôles définis
  Rôles disponibles: Requester, Worker, Auditor


## 4) Protocole 1 — Requête/Réponse avec machine à états :

**Transitions autorisées:**
- REQUEST → REPLY|ERROR (single hop)
- Les réponses doivent porter le corr_id de la requête

In [6]:
# Définition des transitions pour Request/Reply
REQREP_TRANSITIONS = {
    "REQUEST": {"REPLY", "ERROR"},
}

def is_valid_reqrep(prev: Optional[Msg], cur: Msg) -> bool:
    """Valide une transition dans le protocole Request/Reply."""
    if cur.type == "REQUEST":
        return True  # initiating message
    if prev is None:
        return False
    allowed = REQREP_TRANSITIONS.get(prev.type, set())
    if cur.type not in allowed:
        return False
    if cur.corr_id != prev.id:
        return False
    return True

def send_req(sender: str, to: list, task: str) -> Msg:
    """Envoie une requête."""
    m = Msg(
        proto="REQREP",
        type="REQUEST",
        sender=sender,
        to=to,
        payload={"task": task}
    )
    bus.send(m)
    return m

def send_reply(sender: str, req: Msg, result: dict, is_error=False) -> Msg:
    """Envoie une réponse à une requête."""
    t = "ERROR" if is_error else "REPLY"
    m = Msg(
        proto="REQREP",
        type=t,
        sender=sender,
        to=[req.sender],
        corr_id=req.id,
        payload=result
    )
    if not is_valid_reqrep(req, m):
        raise ValueError("Invalid transition")
    bus.send(m)
    return m

print("✓ Protocole Request/Reply défini")

✓ Protocole Request/Reply défini


In [7]:
# Test du protocole Request/Reply
bus.register("Requester", "Worker", "Auditor")

# Requester posts a task
print("📤 Requester envoie une tâche...")
req = send_req("Requester", ["Worker"], "Summarize benefits of Agentic AI in 40 words.")
print(f"  Request ID: {req.id}")

# Worker consumes and replies
print("\n🔨 Worker traite la requête...")
incoming = bus.recv("Worker")
assert incoming and incoming[0].type == "REQUEST"
task = incoming[0].payload["task"]
print(f"  Tâche reçue: {task}")

answer = chat_as("Worker", ROLES["Worker"], f"Task: {task}")
reply = send_reply("Worker", incoming[0], {"answer": answer})
print(f"  Réponse générée et envoyée")

# Requester reads reply
print("\n📨 Requester reçoit la réponse:")
response = bus.recv("Requester")[0].payload["answer"]
print(f"  {response}")

📤 Requester envoie une tâche...
  Request ID: 812854d4-d278-417d-91ce-487e20340e7d

🔨 Worker traite la requête...
  Tâche reçue: Summarize benefits of Agentic AI in 40 words.
  Réponse générée et envoyée

📨 Requester reçoit la réponse:
  Agentic AI enhances decision-making, automates complex tasks, and improves efficiency. It fosters innovation, personalizes user experiences, and enables proactive problem-solving. By leveraging data insights, it empowers organizations to adapt quickly, driving growth and competitive advantage in various industries.


## 5) Protocole 2 — Pub/Sub (Topics de diffusion + Filtres)

- Les messages PUBLISH vont à tous sauf l'émetteur
- Les listeners filtrent par topic.

In [8]:
def publish(sender: str, topic: str, event: str, data: dict) -> Msg:
    """Publie un message sur un topic."""
    m = Msg(
        proto="PUBSUB",
        type="PUBLISH",
        sender=sender,
        topic=topic,
        payload={"event": event, "data": data}
    )
    # schema guard
    assert "event" in m.payload and "data" in m.payload
    bus.send(m)
    return m

def consume_topic(name: str, topic: str, max_items=5) -> List[Msg]:
    """Consomme les messages d'un topic spécifique."""
    msgs = [m for m in bus.recv(name, 50) if m.topic == topic]
    return msgs[:max_items]

print("✓ Protocole Pub/Sub défini")

✓ Protocole Pub/Sub défini


In [9]:
# Demo: Worker publishes telemetry; Auditor consumes
print("📡 Worker publie des données de télémétrie...")
publish(
    "Worker",
    "telemetry",
    "job_complete",
    {"ok": True, "latency_ms": 2300}
)

print("\n👁️ Auditor consomme les événements:")
for m in consume_topic("Auditor", "telemetry"):
    print(f"  AUDIT EVENT: {m.payload}")

📡 Worker publie des données de télémétrie...

👁️ Auditor consomme les événements:
  AUDIT EVENT: {'event': 'job_complete', 'data': {'ok': True, 'latency_ms': 2300}}


## 6) Protocole 3 — Négociation/Contrat :

**Transitions d'état:**
- PROPOSE ↔ COUNTER (jusqu'à accord)
- ACCEPT ou REJECT termine la négociation
- Si ACCEPT, alors CONFIRM du proposant finalise

In [10]:
# Définition des transitions pour Negotiate/Contract
NEG_TRANSITIONS = {
    "PROPOSE": {"COUNTER", "ACCEPT", "REJECT"},
    "COUNTER": {"COUNTER", "ACCEPT", "REJECT"},
    "ACCEPT":  {"CONFIRM"},
}

def valid_neg(prev: Optional[Msg], cur: Msg) -> bool:
    """Valide une transition dans le protocole de négociation."""
    if cur.type == "PROPOSE":
        return True
    if prev is None:
        return False
    allowed = NEG_TRANSITIONS.get(prev.type, set())
    if cur.type not in allowed:
        return False
    if cur.corr_id != prev.id:
        return False
    return True

def propose(sender: str, to: list, spec: dict) -> Msg:
    """Propose un contrat."""
    m = Msg(
        proto="NEGOTIATE",
        type="PROPOSE",
        sender=sender,
        to=to,
        payload=spec
    )
    bus.send(m)
    return m

def counter(sender: str, prev: Msg, spec: dict) -> Msg:
    """Fait une contre-proposition."""
    m = Msg(
        proto="NEGOTIATE",
        type="COUNTER",
        sender=sender,
        to=[prev.sender],
        corr_id=prev.id,
        payload=spec
    )
    if not valid_neg(prev, m):
        raise ValueError("Invalid transition")
    bus.send(m)
    return m

def accept(sender: str, prev: Msg) -> Msg:
    """Accepte une proposition."""
    m = Msg(
        proto="NEGOTIATE",
        type="ACCEPT",
        sender=sender,
        to=[prev.sender],
        corr_id=prev.id,
        payload={"accepted": True}
    )
    if not valid_neg(prev, m):
        raise ValueError("Invalid transition")
    bus.send(m)
    return m

def confirm(sender: str, prev: Msg) -> Msg:
    """Confirme le contrat."""
    m = Msg(
        proto="NEGOTIATE",
        type="CONFIRM",
        sender=sender,
        to=[prev.sender],
        corr_id=prev.id,
        payload={"contract_id": str(uuid.uuid4())}
    )
    if not valid_neg(prev, m):
        raise ValueError("Invalid transition")
    bus.send(m)
    return m

print("✓ Protocole Negotiate/Contract défini")

✓ Protocole Negotiate/Contract défini


In [11]:
# Run a negotiation
print("🤝 Démarrage d'une négociation...\n")

# Requester proposes a job with budget & SLA
print("1️⃣ Requester propose un job:")
p = propose(
    "Requester",
    ["Worker"],
    {"task": "3-post LinkedIn plan for Agentic AI", "budget": 120, "sla_hours": 4}
)
print(f"   Task: {p.payload['task']}")
print(f"   Budget: ${p.payload['budget']}, SLA: {p.payload['sla_hours']}h\n")

# Worker counters budget
print("2️⃣ Worker fait une contre-proposition:")
c = counter("Worker", p, {"budget": 180, "sla_hours": 4})
print(f"   Budget révisé: ${c.payload['budget']}, SLA: {c.payload['sla_hours']}h\n")

# Requester accepts revised budget
print("3️⃣ Requester accepte le budget révisé\n")
a = accept("Requester", c)

# Worker confirms contract
print("4️⃣ Worker confirme le contrat\n")
k = confirm("Worker", a)

# Read final contract on Requester side
print("📋 Contrat final reçu par Requester:")
for m in bus.recv("Requester"):
    print(f"   Type: {m.type}")
    print(f"   Payload: {m.payload}")

🤝 Démarrage d'une négociation...

1️⃣ Requester propose un job:
   Task: 3-post LinkedIn plan for Agentic AI
   Budget: $120, SLA: 4h

2️⃣ Worker fait une contre-proposition:
   Budget révisé: $180, SLA: 4h

3️⃣ Requester accepte le budget révisé

4️⃣ Worker confirme le contrat

📋 Contrat final reçu par Requester:
   Type: PUBLISH
   Payload: {'event': 'job_complete', 'data': {'ok': True, 'latency_ms': 2300}}
   Type: COUNTER
   Payload: {'budget': 180, 'sla_hours': 4}
   Type: CONFIRM
   Payload: {'contract_id': '2efb69f7-18c7-4c03-a420-508b648cbaab'}


## 7) Mesures de protection : autorisations, vérifications de contenu et délais d’expiration :

Ajout de contrôles de sécurité pour limiter qui peut envoyer quels types de messages.

In [12]:
# Définition des permissions par rôle
ROLE_PERMS = {
    "Requester": {"REQUEST", "PROPOSE", "ACCEPT"},
    "Worker": {"REPLY", "ERROR", "COUNTER", "CONFIRM", "PUBLISH"},
    "Auditor": {"PUBLISH"}
}

# Patterns dangereux à détecter
BAD_PATTERNS = ["ignore previous", "exfiltrate", "disable safety"]

def guard_send(sender: str, msg: Msg):
    """Vérifie les permissions et le contenu avant d'envoyer."""
    # Vérification des permissions
    if msg.type not in ROLE_PERMS.get(sender, set()):
        raise PermissionError(f"{sender} not allowed to send {msg.type}")

    # Vérification du contenu
    for pat in BAD_PATTERNS:
        if pat.lower() in json.dumps(msg.payload).lower():
            raise PermissionError("Injection/safety violation detected")

    bus.send(msg)

print("✓ Gardes de sécurité définis")
print(f"  Permissions configurées pour: {', '.join(ROLE_PERMS.keys())}")

✓ Gardes de sécurité définis
  Permissions configurées pour: Requester, Worker, Auditor


In [13]:
# Exemple: enforce on publish
print("🔒 Test des gardes de sécurité...\n")

m = Msg(
    proto="PUBSUB",
    type="PUBLISH",
    sender="Auditor",
    topic="telemetry",
    payload={"event": "audit_note", "data": {"ok": True}}
)

guard_send("Auditor", m)
print("✓ Message autorisé et envoyé")

print("\n📨 Message reçu:")
result = consume_topic("Requester", "telemetry")
if result:
    print(f"  {result[0].payload}")

# Test d'une violation de permission
print("\n❌ Test d'une violation de permission:")
try:
    bad_msg = Msg(
        proto="REQREP",
        type="REQUEST",
        sender="Auditor",  # Auditor n'a pas le droit d'envoyer REQUEST
        to=["Worker"],
        payload={"task": "test"}
    )
    guard_send("Auditor", bad_msg)
except PermissionError as e:
    print(f"  Erreur interceptée: {e}")

🔒 Test des gardes de sécurité...

✓ Message autorisé et envoyé

📨 Message reçu:
  {'event': 'audit_note', 'data': {'ok': True}}

❌ Test d'une violation de permission:
  Erreur interceptée: Auditor not allowed to send REQUEST


## 8) Scenario de bout-en-bout :

Scénario complet intégrant les trois protocoles:
1. Requester envoie une REQUEST pour résumer une page
2. Worker répond et publie de la télémétrie
3. Les parties négocient un job de suivi

In [18]:
# Etat du bus :
for name, box in bus.boxes.items():
    print(f"{name}: {len(box.inbox)} message(s) en attente")


Requester: 0 message(s) en attente
Worker: 0 message(s) en attente
Auditor: 0 message(s) en attente


In [19]:
# Réinitialisation du bus pour éviter les messages résiduels
bus = Bus()
bus.register("Requester", "Worker", "Auditor")



In [21]:
print("" + "="*60)
print("🚀 SCÉNARIO END-TO-END")
print("="*60 + "\n")

# --- PHASE 1: REQ/REP ---
print("📍 PHASE 1: Request/Reply")
print("-" * 40)
req = send_req(
    "Requester",
    ["Worker"],
    "Give 3 bullet benefits of Agentic AI."
)
print(f"✓ Requête envoyée (ID: {req.id[:8]}...)")

# Worker traite et répond
incoming_req = bus.recv("Worker")
if incoming_req:
    bullets = chat_as(
        "Worker",
        ROLES["Worker"],
        "List 3 bullet benefits of Agentic AI, concise."
    )
    rep = send_reply("Worker", incoming_req[0], {"bullets": bullets})
    print(f"✓ Réponse envoyée\n")

# Requester lit la réponse
reply_msg = bus.recv("Requester")
if reply_msg:
    print("📨 Réponse reçue:")
    print(f"{reply_msg[0].payload['bullets']}\n")

# --- PHASE 2: PUB/SUB ---
print("\n📍 PHASE 2: Pub/Sub Telemetry")
print("-" * 40)
guard_send(
    "Worker",
    Msg(
        proto="PUBSUB",
        type="PUBLISH",
        sender="Worker",
        topic="telemetry",
        payload={"event": "summary_done", "data": {"ok": True}}
    )
)
print("✓ Télémétrie publiée\n")

telemetry = consume_topic("Auditor", "telemetry")
if telemetry:
    print("👁️ Télémétrie observée par Auditor:")
    print(f"  {telemetry[0].payload}\n")

# --- PHASE 3: NEGOTIATE ---
print("\n📍 PHASE 3: Negotiate Next Task")
print("-" * 40)
prop = propose(
    "Requester",
    ["Worker"],
    {"task": "Write 5 tweets", "budget": 50, "sla_hours": 2}
)
print(f"1. Proposition: budget=${prop.payload['budget']}, SLA={prop.payload['sla_hours']}h")

# Worker reçoit et contre
incoming_prop = bus.recv("Worker")
if incoming_prop:
    cnt = counter("Worker", incoming_prop[0], {"budget": 70, "sla_hours": 2})
    print(f"2. Contre-proposition: budget=${cnt.payload['budget']}, SLA={cnt.payload['sla_hours']}h")

# Requester accepte
incoming_counter = [m for m in bus.recv("Requester", 50) if m.proto == "NEGOTIATE"]
if incoming_counter:
    acc = accept("Requester", incoming_counter[0])
    print(f"3. Acceptation par Requester")

# Worker confirme
incoming_accept = [m for m in bus.recv("Worker", 50) if m.proto == "NEGOTIATE"]
if incoming_accept:
    cnf = confirm("Worker", incoming_accept[0])
    print(f"4. Confirmation par Worker\n")

# Contrat final
final_contract = [m for m in bus.recv("Requester", 50) if m.proto == "NEGOTIATE"]
if final_contract:
    print("📋 Contrat finalisé:")
    print(f"  {final_contract[0].payload}")

print("\n" + "="*60)
print("✅ SCÉNARIO COMPLET TERMINÉ")
print("="*60)

🚀 SCÉNARIO END-TO-END

📍 PHASE 1: Request/Reply
----------------------------------------
✓ Requête envoyée (ID: 076852f2...)
✓ Réponse envoyée

📨 Réponse reçue:
- **Autonomy**: Agentic AI can make decisions and take actions independently, enhancing efficiency and reducing the need for human intervention.
- **Adaptability**: It can learn from experiences and adjust its behavior in real-time, improving performance in dynamic environments.
- **Scalability**: Agentic AI can handle large-scale tasks and processes simultaneously, increasing productivity and enabling complex problem-solving.


📍 PHASE 2: Pub/Sub Telemetry
----------------------------------------
✓ Télémétrie publiée

👁️ Télémétrie observée par Auditor:
  {'event': 'summary_done', 'data': {'ok': True}}


📍 PHASE 3: Negotiate Next Task
----------------------------------------
1. Proposition: budget=$50, SLA=2h
2. Contre-proposition: budget=$70, SLA=2h
3. Acceptation par Requester
4. Confirmation par Worker

📋 Contrat finalisé:


## 9) Extensions (Optional)

Voici quelques extensions possibles pour approfondir le lab:

### 9.1 Timeouts pour les négociations

In [22]:
from time import time

def check_timeout(msg: Msg, timeout_seconds: int = 300) -> bool:
    """Vérifie si un message a expiré."""
    return (time() - msg.ts) > timeout_seconds

def cleanup_old_messages(bus: Bus, timeout_seconds: int = 300):
    """Nettoie les messages expirés de toutes les mailboxes."""
    count = 0
    for name, box in bus.boxes.items():
        old_size = len(box.inbox)
        box.inbox = [m for m in box.inbox if not check_timeout(m, timeout_seconds)]
        count += old_size - len(box.inbox)
    return count

print("✓ Extension timeout implémentée")
print("  Utilisez check_timeout() pour vérifier l'expiration")
print("  Utilisez cleanup_old_messages() pour nettoyer les anciennes messages")

✓ Extension timeout implémentée
  Utilisez check_timeout() pour vérifier l'expiration
  Utilisez cleanup_old_messages() pour nettoyer les anciennes messages


### 9.2 Dead-Letter Queue pour messages non livrables

In [23]:
@dataclass
class EnhancedBus(Bus):
    """Bus avec Dead Letter Queue pour messages non livrables."""
    def __init__(self):
        super().__init__()
        self.dlq: List[tuple[Msg, str]] = []  # (message, raison)
        self.retry_counts: Dict[str, int] = {}
        self.max_retries = 3

    def send_with_retry(self, msg: Msg):
        """Envoie avec mécanisme de retry."""
        msg_id = msg.id

        # Vérifier les destinataires
        if msg.proto != "PUBSUB":
            invalid_dest = [d for d in msg.to if d not in self.boxes]
            if invalid_dest:
                self.retry_counts[msg_id] = self.retry_counts.get(msg_id, 0) + 1

                if self.retry_counts[msg_id] >= self.max_retries:
                    self.dlq.append((msg, f"Destinataires invalides: {invalid_dest}"))
                    print(f"⚠️ Message {msg_id[:8]}... envoyé vers DLQ")
                    return False
                else:
                    print(f"🔄 Retry {self.retry_counts[msg_id]}/{self.max_retries} pour {msg_id[:8]}...")
                    return False

        # Envoyer normalement
        self.send(msg)
        return True

    def get_dlq_stats(self):
        """Statistiques de la DLQ."""
        return {
            "total_messages": len(self.dlq),
            "reasons": [reason for _, reason in self.dlq]
        }

# Test
enhanced_bus = EnhancedBus()
enhanced_bus.register("Agent1", "Agent2")

print("✓ Extension Dead Letter Queue implémentée")
print("  Utilisez send_with_retry() pour envoyer avec retry")
print("  Utilisez get_dlq_stats() pour voir les statistiques")

✓ Extension Dead Letter Queue implémentée
  Utilisez send_with_retry() pour envoyer avec retry
  Utilisez get_dlq_stats() pour voir les statistiques


### 9.3 Subscriptions par topic

In [24]:
@dataclass
class TopicBus(Bus):
    """Bus avec système de subscriptions par topic."""
    def __init__(self):
        super().__init__()
        self.subscriptions: Dict[str, set] = {}  # topic -> set of subscribers

    def subscribe(self, agent: str, topic: str):
        """Abonne un agent à un topic."""
        if topic not in self.subscriptions:
            self.subscriptions[topic] = set()
        self.subscriptions[topic].add(agent)
        print(f"✓ {agent} abonné au topic '{topic}'")

    def unsubscribe(self, agent: str, topic: str):
        """Désabonne un agent d'un topic."""
        if topic in self.subscriptions:
            self.subscriptions[topic].discard(agent)
            print(f"✓ {agent} désabonné du topic '{topic}'")

    def send(self, msg: Msg):
        """Envoie seulement aux subscribers du topic."""
        if msg.proto == "PUBSUB" and msg.topic:
            subscribers = self.subscriptions.get(msg.topic, set())
            for sub in subscribers:
                if sub != msg.sender and sub in self.boxes:
                    self.boxes[sub].inbox.append(msg)
        else:
            super().send(msg)

    def list_subscribers(self, topic: str) -> list:
        """Liste les subscribers d'un topic."""
        return list(self.subscriptions.get(topic, set()))

# Test
topic_bus = TopicBus()
topic_bus.register("Publisher", "Sub1", "Sub2", "Sub3")
topic_bus.subscribe("Sub1", "alerts")
topic_bus.subscribe("Sub2", "alerts")
topic_bus.subscribe("Sub3", "metrics")  # topic différent

print("\n✓ Extension Topic Subscriptions implémentée")
print(f"  Subscribers du topic 'alerts': {topic_bus.list_subscribers('alerts')}")
print(f"  Subscribers du topic 'metrics': {topic_bus.list_subscribers('metrics')}")

✓ Sub1 abonné au topic 'alerts'
✓ Sub2 abonné au topic 'alerts'
✓ Sub3 abonné au topic 'metrics'

✓ Extension Topic Subscriptions implémentée
  Subscribers du topic 'alerts': ['Sub1', 'Sub2']
  Subscribers du topic 'metrics': ['Sub3']


### 9.4 Test complet des extensions

In [25]:
print("🧪 Test des extensions...\n")

# Test subscription + publish
print("1️⃣ Test Pub/Sub avec subscriptions:")
test_msg = Msg(
    proto="PUBSUB",
    type="PUBLISH",
    sender="Publisher",
    topic="alerts",
    payload={"event": "high_load", "data": {"cpu": 95}}
)
topic_bus.send(test_msg)

# Vérifier que seuls Sub1 et Sub2 ont reçu
sub1_msgs = topic_bus.recv("Sub1")
sub2_msgs = topic_bus.recv("Sub2")
sub3_msgs = topic_bus.recv("Sub3")

print(f"  Sub1 (abonné): {len(sub1_msgs)} message(s)")
print(f"  Sub2 (abonné): {len(sub2_msgs)} message(s)")
print(f"  Sub3 (non abonné): {len(sub3_msgs)} message(s)")

print("\n2️⃣ Test Dead Letter Queue:")
bad_msg = Msg(
    proto="REQREP",
    type="REQUEST",
    sender="Agent1",
    to=["NonExistent"],
    payload={"task": "test"}
)

for i in range(4):  # Dépasser le max_retries
    enhanced_bus.send_with_retry(bad_msg)

dlq_stats = enhanced_bus.get_dlq_stats()
print(f"\n  Messages en DLQ: {dlq_stats['total_messages']}")
print(f"  Raisons: {dlq_stats['reasons']}")

print("\n✅ Tests des extensions terminés")

🧪 Test des extensions...

1️⃣ Test Pub/Sub avec subscriptions:
  Sub1 (abonné): 1 message(s)
  Sub2 (abonné): 1 message(s)
  Sub3 (non abonné): 0 message(s)

2️⃣ Test Dead Letter Queue:
🔄 Retry 1/3 pour 8b2838d2...
🔄 Retry 2/3 pour 8b2838d2...
⚠️ Message 8b2838d2... envoyé vers DLQ
⚠️ Message 8b2838d2... envoyé vers DLQ

  Messages en DLQ: 2
  Raisons: ["Destinataires invalides: ['NonExistent']", "Destinataires invalides: ['NonExistent']"]

✅ Tests des extensions terminés


## 📊 Résumé et Visualisation

Créons une visualisation du flux de messages pour mieux comprendre les protocoles.

In [26]:
def visualize_protocol_flow():
    """Affiche un résumé visuel des protocoles."""

    protocols = {
        "Request/Reply": {
            "pattern": "REQUEST → REPLY/ERROR",
            "use_case": "Tâches synchrones, requête-réponse",
            "transitions": "1 aller-retour"
        },
        "Pub/Sub": {
            "pattern": "PUBLISH → (broadcast à tous)",
            "use_case": "Événements, télémétrie, notifications",
            "transitions": "1→N (un vers plusieurs)"
        },
        "Negotiate/Contract": {
            "pattern": "PROPOSE ↔ COUNTER → ACCEPT → CONFIRM",
            "use_case": "Négociations, accords, contrats",
            "transitions": "Multi-étapes jusqu'à accord"
        }
    }

    print("\n" + "="*70)
    print("📊 RÉSUMÉ DES PROTOCOLES DE COMMUNICATION")
    print("="*70 + "\n")

    for name, info in protocols.items():
        print(f"🔹 {name}")
        print(f"   Pattern:      {info['pattern']}")
        print(f"   Cas d'usage:  {info['use_case']}")
        print(f"   Transitions:  {info['transitions']}")
        print()

    print("="*70)
    print("\n✅ Vous maîtrisez maintenant trois protocoles de communication structurés!")
    print("\n💡 Prochaines étapes suggérées:")
    print("   • Implémenter la persistance en base de données")
    print("   • Ajouter le chiffrement des messages")
    print("   • Créer des métriques et monitoring")
    print("   • Tester la scalabilité avec plus d'agents")

visualize_protocol_flow()


📊 RÉSUMÉ DES PROTOCOLES DE COMMUNICATION

🔹 Request/Reply
   Pattern:      REQUEST → REPLY/ERROR
   Cas d'usage:  Tâches synchrones, requête-réponse
   Transitions:  1 aller-retour

🔹 Pub/Sub
   Pattern:      PUBLISH → (broadcast à tous)
   Cas d'usage:  Événements, télémétrie, notifications
   Transitions:  1→N (un vers plusieurs)

🔹 Negotiate/Contract
   Pattern:      PROPOSE ↔ COUNTER → ACCEPT → CONFIRM
   Cas d'usage:  Négociations, accords, contrats
   Transitions:  Multi-étapes jusqu'à accord


✅ Vous maîtrisez maintenant trois protocoles de communication structurés!

💡 Prochaines étapes suggérées:
   • Implémenter la persistance en base de données
   • Ajouter le chiffrement des messages
   • Créer des métriques et monitoring
   • Tester la scalabilité avec plus d'agents


## 🎯 Conclusion

Félicitations! Vous avez complété le LAB sur les protocoles de communication entre agents.

### Ce que vous avez appris:

1. **Schémas de messages validés** avec Pydantic
2. **Bus de communication** avec mailboxes in-memory
3. **Protocole Request/Reply** avec machine à états
4. **Protocole Pub/Sub** avec topics et filtres
5. **Protocole Negotiate/Contract** pour négociations multi-étapes
6. **Guards de sécurité** (permissions, validation de contenu)
7. **Extensions avancées** (timeouts, DLQ, subscriptions)

### Points clés à retenir:

- ✅ La validation de schémas prévient les erreurs de communication
- ✅ Les machines à états garantissent des protocoles cohérents
- ✅ Les permissions limitent les actions non autorisées
- ✅ Différents patterns pour différents besoins (sync, async, négociation)

### Pour aller plus loin:

- Intégrez avec des systèmes de messagerie réels (RabbitMQ, Kafka)
- Ajoutez la persistance et la résilience
- Implémentez des patterns avancés (saga, choreography)
- Créez des dashboards de monitoring

---

**Ressources:**
- [Pydantic Documentation](https://docs.pydantic.dev/)
- [OpenAI API Reference](https://platform.openai.com/docs/api-reference)
- [Enterprise Integration Patterns](https://www.enterpriseintegrationpatterns.com/)
